# Exploration

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

raw_data = pd.read_csv('raw_merged.csv')


C:\Users\shery\AppData\Local\Temp\ipykernel_14140\2723244342.py:5: DtypeWarning: Columns (1,7,21,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('raw_merged.csv')


In [ ]:
raw_data = raw_data.replace(['EX - Excellent'] , 'E - Excellent')
raw_data["case_created_date"] = raw_data["case_created_date"][:11]
raw_data["close_date"] = raw_data["close_date"][:11]
raw_data["last_case_update"] = raw_data["last_case_update"][:11]

#split data insto 70 train, 15 val, 15 test
train_data, temp_data = train_test_split(raw_data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print("Train data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (2164743, 31)
Validation data shape: (463874, 31)
Test data shape: (463874, 31)


In [ ]:
print(train_data['overall_cond'].value_counts())
print('NA COUNTS: ', train_data.isna().sum())
print('num sam ids: ', train_data['sam_id'].nunique())


overall_cond
A - Average       1798955
G - Good           111181
E - Excellent       30844
VG - Very Good       7101
F - Fair              689
P - Poor               12
Name: count, dtype: int64
NA COUNTS:  Unnamed: 0                 0
over_5               1976368
year                       0
full_address               0
sam_id                     0
building_id                0
ward_id                    0
parcel_num                 0
case_created_date          5
last_case_update           0
targeted_deadline     303630
close_date            152674
case_met_deadline      11900
case_status            11893
closure_reason         11893
case_title             11893
case_subject           11893
case_reason            11893
case_type              11893
case_department        11893
p_id                  191098
cm_id                 247421
gis_id                191320
landlord_name         191168
int_cond              608429
ext_cond              568442
overall_cond          215961
bdrm_cond 

In [7]:
train_data.head()
train_data['overall_cond'].head()

411370     A - Average
1985085    A - Average
1482786    A - Average
2112011            NaN
495677     A - Average
Name: overall_cond, dtype: object

# Additional Column(s)

In [ ]:
def check(cond):
        if cond == 'F - Fair' or cond == 'P - Poor':
            return 1
        else:
            return 0
train_data['prob_address'] = train_data['overall_cond'].apply(check)


print(train_data['overall_cond'].value_counts())
print(train_data['prob_address'].value_counts())

overall_cond
A - Average       1798955
G - Good           111181
E - Excellent       30844
VG - Very Good       7101
F - Fair              689
P - Poor               12
Name: count, dtype: int64
prob_address
0    2164042
1        701
Name: count, dtype: int64


# TRAIN TEST SPLIT!!!

In [11]:
sample_size = 0.2
print("Taking smaller sample size for faster processing...")

train_data_with_target = train_data[train_data["prob_address"].notna()].copy()

from sklearn.model_selection import train_test_split
_, train_data_sample = train_test_split(
    train_data_with_target,
    test_size = sample_size, 
    random_state=42, 
    stratify=train_data_with_target["prob_address"] 
)

print("Sampled training set shape: ", train_data_sample.shape)
print("Sampled score distribution", train_data_sample["prob_address"].value_counts())



Taking smaller sample size for faster processing...
Sampled training set shape:  (432949, 32)
Sampled score distribution prob_address
0    432809
1       140
Name: count, dtype: int64


In [20]:
print(train_data["heat_type"].unique())

[nan 'F - Forced Hot Air' 'W - Ht Water/Steam' 'P - Heat Pump'
 'E - Electric' 'S - Space Heat' 'N - None' 'O - Other']


# Feature Engineering 

In [ ]:
#feature engineering function
def create_features(dataset):

    #Feature 0: Fix Values 
    
    # Fill in Nans 
    dataset["case_title"] = dataset["case_title"].fillna("").astype(str)
    dataset["case_subject"] = dataset["case_subject"].fillna("").astype(str)
    dataset["case_reason"] = dataset["case_reason"].fillna("").astype(str)
    dataset["over_five"] = dataset["over_five"].fillna("").astype(str)
    dataset["int_cond"] = dataset["int_cond"].fillna("").astype(str)
    dataset["ext_cond"] = dataset["ext_cond"].fillna("").astype(str)
    dataset["overall_cond"] = dataset["overall_cond"].fillna("").astype(str)
    dataset["bdrm_cond"] = dataset["bdrm_cond"].fillna("").astype(str)
    dataset["heat_type"] = dataset["heat_type"].fillna("").astype(str)
    dataset["ac_type"] = dataset["ac_type"].fillna("").astype(str)
    
    # String -> Integer 
    dataset["year"] = dataset["year"].astype(int)
    dataset["parcel_num"] = dataset["parcel_num"].astype(int)
    dataset["cm_id"] = dataset["cm_id"].astype(int)

    # Standardize Values
    dataset["over_5"] = dataset["over_5"].replace(["N", "No"], "NO")
    dataset["over_5"] = dataset["over_5"].replace(["Y"], "YES")
    dataset["over_5"] = dataset["over_5"].replace(["nan"], "")
    
    #Feature 1: Text Features 
    print("Extracting text features...")
    dataset["case_reason_length"] = dataset["case_reason"].fillna('').astype(str).str.len()
    dataset['case_reason_word_count'] = dataset['case_reason'].fillna('').astype(str).str.split().str.len()

    dataset['case_title_length'] = dataset['case_title'].fillna('').astype(str).str.len()
    dataset['case_title_word_count'] = dataset['case_title'].fillna('').astype(str).str.split().str.len()

    dataset['case_type_length'] = dataset['case_type'].fillna('').astype(str).str.len()
    dataset['case_type_word_count'] = dataset['case_type'].fillna('').astype(str).str.split().str.len()

    #Feature 2: Time Based Features 

    


In [ ]:
#apply feature engineering 

dataset = train_data_sample.copy() 